In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import copy

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

%load_ext autoreload
%autoreload 2

In [2]:
mydateparser = lambda x: datetime.strptime(x, "%Y-%m-%d")
info = pd.read_csv('../data/snp_info.csv', index_col=0)

In [3]:
stocks_analysis = ['KO', 'TGT', 'PFE', 'MSFT', 'CVX', 'DVN', 'DAL', 'JPM', 'PEP', 'AAPL']

In [4]:
stocks = pd.read_csv('stocks_by_sector.csv')
stocks

,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology
0,T,F,KO,MRO,BAC,PFE,GE,AAPL
1,TWTR,GM,KR,KMI,WFC,MRK,DAL,AMD
2,FB,EBAY,PG,XOM,C,GILD,CSX,MU
3,CMCSA,SBUX,WMT,HAL,RF,BMY,AAL,MSFT
4,VZ,NKE,MDLZ,WMB,JPM,BSX,LUV,INTC
5,NFLX,M,MO,COP,KEY,ABT,FAST,CSCO
6,DIS,MGM,COTY,SLB,MS,CVS,CAT,HPE
7,ATVI,TJX,WBA,DVN,HBAN,ABBV,JCI,ORCL
8,IPG,TGT,PM,CVX,SCHW,JNJ,UAL,NVDA
9,DISCA,NWL,PEP,COG,SYF,MDT,UNP,AMAT


In [5]:
idx_dict = {}
for i in range(len(stocks.index)):
    for j in range(len(stocks.columns)):
        stock = stocks.iloc[i,j]
        idx_dict[stock] = (i,j)

---------
Get Samples and Labels
---------

In [6]:
samples = pd.read_csv('samples_sector.csv', index_col=0, header=[0,1,2], parse_dates=True, date_parser=mydateparser)
samples.head()

Sector     Communication Services                                             \
Asset                     asset_1                                              
Metrics                     Close        RSI  k_percent  r_percent      MACD   
Date                                                                           
2016-01-22              35.139999  66.984998  97.740062  -2.259938  0.092199   
2016-01-25              35.000000  63.461411  78.712861 -21.287139  0.123844   
2016-01-26              35.400002  68.861369  92.129714  -7.870286  0.170558   
2016-01-27              35.480000  69.888295  86.610888 -13.389112  0.209019   
2016-01-28              35.529999  70.587814  88.702907 -11.297093  0.239060   

Sector                                                                  \
Asset                                                          asset_2   
Metrics    MACD_EMA9 Price Rate Of Change On Balance Volume      Close   
Date                                                                     
2016-01-22  0.021266             0.047704          10592300  17.840000   
2016-01-25  0.042529             0.030928         -19672100  17.020000   
2016-01-26  0.068877             0.044248          22005000  17.010000   
2016-01-27  0.097551             0.051571          72268000  16.780001   
2016-01-28  0.126372             0.035860          98452400  16.490000   

Sector                       ...        Information Technology  \
Asset                        ...                       asset_9   
Metrics           RSI        ...          Price Rate Of Change   
Date                         ...                                 
2016-01-22  30.079617        ...                     -0.039824   
2016-01-25  23.781919        ...                     -0.042453   
2016-01-26  23.712060        ...                     -0.049039   
2016-01-27  21.997274        ...                     -0.030759   
2016-01-28  19.903236        ...                     -0.021625   

Sector                                                                    \
Asset                          asset_10                                    
Metrics    On Balance Volume      Close        RSI  k_percent  r_percent   
Date                                                                       
2016-01-22          -9242700  17.049999  51.119297  43.165445 -56.834555   
2016-01-25         -16010900  16.910000  47.801520  38.129494 -61.870506   
2016-01-26         -10312300  17.350000  57.746426  63.025232 -36.974768   
2016-01-27         -16104800  17.090000  51.107526  70.454523 -29.545477   
2016-01-28         -23024200  16.680000  42.266121  47.976887 -52.023113   

Sector                                                                 
Asset                                                                  
Metrics         MACD MACD_EMA9 Price Rate Of Change On Balance Volume  
Date                                                                   
2016-01-22 -0.097708 -0.124555             0.015485         -82509700  
2016-01-25 -0.078060 -0.114917             0.013789         -91721800  
2016-01-26 -0.035484 -0.098570             0.035821         -75935100  
2016-01-27 -0.018663 -0.082221             0.054938         -88525300  
2016-01-28 -0.031403 -0.071871            -0.007733        -106022400  

[5 rows x 640 columns]

In [7]:
labels = pd.read_csv('labels_sector.csv', index_col='Date', parse_dates=True, date_parser=mydateparser)
labels.head()

,T,TWTR,FB,CMCSA,VZ,NFLX,DIS,ATVI,IPG,DISCA,...,AAPL,AMD,MU,MSFT,INTC,CSCO,HPE,ORCL,NVDA,AMAT
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-22,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
2016-01-25,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-26,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-27,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-28,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------------------------------
Training and Prediction (Base)
------------------------------------

In [8]:
from GRTEL.classification import MultiClassifier
from GRTEL.utils import downturn_confidence, print_scores, print_1_percentage, confusion_matrix_metrics 

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [9]:
X = samples
y = np.array(labels)


# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# test_size = 0.25
# k = int(len(X) * (1. - test_size))

# X_train, X_test = X[:k], X[k:]
# y_train, y_test = y[:k], y[k:]

In [10]:
# Initialise classifier
n_classes = y.shape[1]

multi_clf = MultiClassifier(n_classes)

# Train classifer
multi_clf.fit(X_train, y_train)

0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 56 - 57 - 58 - 59 - 60 - 61 - 62 - 63 - 64 - 65 - 66 - 67 - 68 - 69 - 70 - 71 - 72 - 73 - 74 - 75 - 76 - 77 - 78 - 79 - 


In [11]:
# train accuracy
train_score = multi_clf.score(X_train, y_train)
train_accuracy_df = pd.DataFrame(np.array(train_score).reshape(stocks.shape[1], stocks.shape[0])).T
train_accuracy_df.columns = stocks.columns

# test accuracy
test_score = multi_clf.score(X_test, y_test)
test_accuracy_df = pd.DataFrame(np.array(test_score).reshape(stocks.shape[1], stocks.shape[0])).T
test_accuracy_df.columns = stocks.columns

# confusion matrices
conf_matrices = multi_clf.confusion_matrices(X_test, y_test)
conf_matrices = np.array(conf_matrices).reshape(stocks.shape[1], stocks.shape[0], 2, 2)



## Print performance results
results = pd.DataFrame(columns=stocks_analysis, index=['train acc', 'test acc', 'precision', 'recall', 'downturn_precision', 'specificity'], dtype=float)
for stock in stocks_analysis:
    idx = idx_dict[stock]
    results[stock]['train acc'] = train_accuracy_df.iloc[idx]
    results[stock]['test acc'] = test_accuracy_df.iloc[idx]
    
    conf_matrix = conf_matrices[idx[1],idx[0]]
    accuracy, precision, recall, specificity, downturn_precision = confusion_matrix_metrics(conf_matrix)
    
    results[stock]['precision'] = precision
    results[stock]['recall'] = recall
    results[stock]['downturn_precision'] = downturn_precision
    results[stock]['specificity'] = specificity
    
results

,KO,TGT,PFE,MSFT,CVX,DVN,DAL,JPM,PEP,AAPL
train acc,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
test acc,0.814208,0.786885,0.792350,0.732240,0.775956,0.770492,0.852459,0.792350,0.693989,0.830601
precision,0.800000,0.778947,0.847826,0.782609,0.850575,0.775510,0.869565,0.846939,0.721154,0.861386
recall,0.880000,0.804348,0.764706,0.789474,0.725490,0.791667,0.842105,0.783019,0.735294,0.836538
downturn_precision,0.835616,0.795455,0.736264,0.647059,0.708333,0.764706,0.835165,0.729412,0.658228,0.792683
specificity,0.734940,0.769231,0.827160,0.637681,0.839506,0.747126,0.863636,0.805195,0.641975,0.822785


In [12]:
#Scores
score = multi_clf.score(X_train, y_train)
print("\nClassification accuracy (Train):")
print_scores(score); print()

score = multi_clf.score(X_test, y_test)
print("Classification accuracy (Test):")
print_scores(score); print()

print("Percentage of 1s (Test):")
print_1_percentage(y_test, n_classes); print()


Classification accuracy (Train):
[100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%]

Classification accuracy (Test):
[80.87%, 80.33%, 72.13%, 69.40%, 75.41%, 77.05%, 78.69%, 77.05%, 78.69%, 78.14%, 74.86%, 74.86%, 73.22%, 79.78%, 80.33%, 73.22%, 72.13%, 73.77%, 78.69%, 76.50%, 81.42%, 72.13%, 74.86%, 75.96%, 74.86%, 72.68%, 80.